# Semianr 2 - Combinatoris, Graph, Boolean Logic

## Database Repair

In [42]:
import pandas as pd
import numpy as np

### Example 1 (Requirements)

In [43]:
# From Seminar 1
df_resources = pd.DataFrame([[1, "A"], [2, "B"], [3, "C"],
                             [4, "D"],[5, "E"]], columns=["ID", "Name"])

df_projects = pd.DataFrame([[1, "a", "10.05.2020", "15.05.2020"], [2, "b", "13.05.2020", "27.07.2020"],
                            [3, "c", "08.07.2020", "30.07.2020"], [4, "d", "11.12.2020", "29.12.2020"],
                            [5, "e", "06.11.2020", "07.11.2020"]], 
                           columns=["ID", "Name", "Start Time", "End Time"])

df_expertise = pd.DataFrame([[1, 1, 1], [2, 5, 3], [3, 2, 4], [4, 4, 5], [5, 3, 2], [6, 2, 1], [7, 3, 1], [8, 2, 2]],
                           columns=["ID", "ID_res", "ID_pro"])

In [44]:
candidate_solution = [1, 2, 2, 3, 4]

### Example 2 (Requirements)

In [ ]:
# From HW 2
df_resources = pd.DataFrame([[1, "A"], [2, "B"], [3, "C"],
                             [4, "D"]], columns=["ID", "Name"])

df_projects = pd.DataFrame([[1, "a", "10.05.2020", "15.05.2020"], [2, "b", "13.05.2020", "27.07.2020"],
                            [3, "c", "01.07.2020", "20.07.2020"], [4, "d", "25.07.2020", "10.09.2020"]], 
                           columns=["ID", "Name", "Start Time", "End Time"])

df_expertise = pd.DataFrame([[1, 1, 1], [2, 2, 1], [3, 1, 2], [4, 3, 2], 
                             [5, 4, 2], [6, 2, 3], [7, 3, 3], [8, 4, 3], [9, 3, 4]],
                           columns=["ID", "ID_res", "ID_pro"])

In [ ]:
candidate_solution = [1, 1, 2, 3]

### All Feasible Solutions

In [45]:
feasible_solutions = {}

for project_index, project_row in df_projects.iterrows():
    project_id = project_row["ID"]
    res_ids = df_expertise[df_expertise['ID_pro'] == project_id]["ID_res"]
    feasible_solutions[project_id] = []
    
    for res_id in res_ids:
        feasible_solutions[project_id].append(res_id)
        
feasible_solutions

{1: [1, 2, 3], 2: [3, 2], 3: [5], 4: [2], 5: [4]}

### Brute Force - DFS

In [46]:
def date_to_sec(date):
    return time.mktime(datetime.datetime.strptime(date, "%d.%m.%Y").timetuple())

In [47]:
import time
import datetime

# keys = Project IDsdf_expertise
keys = list(feasible_solutions.keys())

def is_valid_solution(solution):
    
    for i in range(len(keys)-1):
        for j in range(i+1, len(keys)):
                
            if solution[i] == solution[j]:
                start_i = date_to_sec(df_projects["Start Time"][i])
                end_i = date_to_sec(df_projects["End Time"][i])
                
                start_j = date_to_sec(df_projects["Start Time"][j])
                end_j = date_to_sec(df_projects["End Time"][j])
                
                if start_i <= start_j:
                    if end_i >= start_j:
                        return False
                    
                elif start_i >= start_j:
                    if end_j >= start_i:
                        return False
    
    return True

# Checking a single solution
#print(is_valid_solution([1, 2, 5, 3, 4]))

#print(is_valid_solution([1, 2, 2, 3, 4]))

### First Solution

In this step we are trying to find the solution by DFS algorithm. DFS tries to build possile solution (tree) based on our feasible_solutions dictionary which contains projects ID (keys) and resourses ID (values).

In the followig cell we are just trying to find the first valid solution.

In [48]:
answer = None

def recursive(solution=[], depth=0):    
    global answer
    #print(solution, depth)
    if(depth >= len(keys)):    
        if(is_valid_solution(solution)):
            #print(', '.join(solution))
            answer = solution
        return
    
    values = feasible_solutions[keys[depth]]
    for v in values:
        if answer:
            return answer
        next_solution = solution.copy()
        next_solution.append(str(v))
        recursive(next_solution, depth+1)
        
recursive()

['1', '3', '5', '2', '4']

### All Solutions

In this step we are trying to traverse all possible combination of the try to print all possible solutions for our problem.

In [49]:
all_valid_sol = []

def recursive(solution=[], depth=0):    
    #print(solution, depth)
    if(depth >= len(keys)):    
        if(is_valid_solution(solution)):
            
            sol_int = [int(x) for x in solution]
            all_valid_sol.append(sol_int)
            
            print(', '.join(solution))
        return
    
    values = feasible_solutions[keys[depth]]
    for v in values:
        next_solution = solution.copy()
        next_solution.append(str(v))
        recursive(next_solution, depth+1)
        
recursive()

1, 3, 5, 2, 4
1, 2, 5, 2, 4
2, 3, 5, 2, 4
3, 2, 5, 2, 4


In [50]:
all_valid_sol

[[1, 3, 5, 2, 4], [1, 2, 5, 2, 4], [2, 3, 5, 2, 4], [3, 2, 5, 2, 4]]

Try to fix the database which we have in **candidate_solution**

In [51]:
def get_distance_list(candidate_sol, all_valid_sol):
    
    ### YOUR CODE ###
    distances = []
    for sol in all_valid_sol:
        dist = list(np.array(candidate_sol) - np.array(sol))
        distances.append(dist)
    #################
    
    return distances

In [52]:
def fix_database(candidate_sol, all_valid_sol):
    
    ### YOUR CODE ###    
    distance_list = get_distance_list(candidate_solution, all_valid_sol)
    print(f"Distance list: {distance_list}\n")
    
    distances = [np.count_nonzero(dist) for dist in distance_list]
    less_dist_index = np.argmin(distances)

    print(f"Closest solution (Database): {all_valid_sol[less_dist_index]}")
    print(f"Candidate solution: {candidate_sol}")
    #################
    
    return all_valid_sol[less_dist_index]

In [53]:
fixed_data = fix_database(candidate_solution, all_valid_sol)
fixed_data

Distance list: [[0, -1, -3, 1, 0], [0, 0, -3, 1, 0], [-1, -1, -3, 1, 0], [-2, 0, -3, 1, 0]]

Closest solution (Database): [1, 2, 5, 2, 4]
Candidate solution: [1, 2, 2, 3, 4]


[1, 2, 5, 2, 4]

In [54]:
res_id = [i for i in range(1, len(fixed_data)+1)]
data_ = [[res_id[i], fixed_data[i]] for i in range(0, len(fixed_data))]

df_fixed_data = pd.DataFrame(data_, columns=["ID_Proj", "ID_Res"])
df_fixed_data

,ID_Proj,ID_Res
0,1,1
1,2,2
2,3,5
3,4,2
4,5,4
